In [ ]:
from model import MiniCharGPTLM
from tokenizer import CharTokenizer
import torch

model_path = "model.pth"
ckpt = torch.load(model_path)

tokenizer = CharTokenizer()
model = MiniCharGPTLM(seqlen=ckpt["seqlen"],
                      h_dim=ckpt["seqlen"],
                      ff_dim=ckpt["ff_dim"],
                      n_head=ckpt["n_head"],
                      n_block=ckpt["n_block"],
                      n_token=len(tokenizer.char2id))

model.load_state_dict(ckpt["state_dict"])

In [ ]:
length = 10
text = "j"
for l in range(length):
    inputs = torch.tensor(text)
    inputs.unsqueeze(0)

    model.eval()
    with torch.no_grad():
        out = model.forward(inputs)
    out = out.argmax(-1).cpu().item()
    next_char = tokenizer.id2char[out]
    text += next_char

In [ ]:
print(text)